In [1]:
##Packages
import time
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import xarray as xr
import datetime as dt
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV,train_test_split 
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor,MLPClassifier
from sklearn.ensemble import RandomForestClassifier ,RandomForestRegressor
from sklearn.preprocessing import scale , StandardScaler
import tarfile
import onedrivesdk
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import onedrivesdk
import urllib.request
from tqdm import tqdm
import scipy
import h5netcdf

In [4]:
tar = tarfile.open("C:/Users/Alexia/OneDrive - insa-toulouse.fr/2D_arome_test.tar.gz")
tar.extractall("D:/DATA")
tar.close()

In [2]:
fname = "C:/Users/Alexia/OneDrive - insa-toulouse.fr/2D_arome_2017.tar.gz"
data = xr.open_dataset(fname)

ValueError: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'h5netcdf', 'scipy']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
http://xarray.pydata.org/en/stable/getting-started-guide/installing.html
http://xarray.pydata.org/en/stable/user-guide/io.html

In [19]:
coords_path = 'C:/Users/Alexia/Desktop/Data/Other/stations_coordinates.csv' #Perso

xtrain_obs_path = 'C:/Users/Alexia/Desktop/Data/Train/X_station_train.csv' #Perso
xtest_obs_path = 'C:/Users/Alexia/Desktop/Data/Test/X_station_test.csv' #Perso

bltest_obs_path = 'C:/Users/Alexia/Desktop/Data/Test/Baselines/Baseline_observation_test.csv' #Perso
bltrain_for_path = 'C:/Users/Alexia/Desktop/Data/Train/Baselines/Baseline_forecast_train.csv' #Perso
bltest_for_path = 'C:/Users/Alexia/Desktop/Data/Test/Baselines/Baseline_forecast_test.csv' #Perso

ytrain_path = 'C:/Users/Alexia/Desktop/Data/Train/Y_train.csv' #Perso

In [18]:
# ##Données
# coords_path = 'C:/Users/Alexia/Desktop/Data/Other/stations_coordinates.csv' #Perso
# xtrainS_path = 'C:/Users/Alexia/Desktop/Data/Train/X_station_train.csv' #Perso
# ytrain_path = 'C:/Users/Alexia/Desktop/Data/Train/Y_train.csv' #Perso
# xtestS_path = 'C:/Users/Alexia/Desktop/Data/Test/X_station_test.csv' #Perso
# bl_obsT_path = 'C:/Users/Alexia/Desktop/Data/Test/Baselines/Baseline_observation_test.csv' #Perso

In [20]:
##Coords
coords = pd.read_csv(coords_path)
display(coords)
print('Missing Data :', coords.isna().sum().sum(), '\n', 'Shape is :', coords.shape)

,number_sta,lat,lon,height_sta
0,86118001,46.477,0.985,120.0
1,86149001,46.917,0.025,60.0
2,56081003,48.050,-3.660,165.0
3,53215001,47.790,-0.710,63.0
4,22135001,48.550,-3.380,148.0
...,...,...,...,...
320,86137003,47.035,0.098,96.0
321,86165005,46.412,0.841,153.0
322,86273001,46.464,1.042,121.0
323,91200002,48.526,1.993,116.0


Missing Data : 0 
 Shape is : (325, 4)


In [21]:
xtrain_obs = pd.read_csv(xtrain_obs_path,parse_dates=['date'],infer_datetime_format=True)
xtrain_obs['number_sta'] = xtrain_obs['number_sta'].astype('category')  
display(xtrain_obs)
print('Missing Data :', xtrain_obs.isna().sum().sum(), '\n', 'Shape is :', xtrain_obs.shape)

,number_sta,date,ff,t,td,hu,dd,precip,Id
0,14066001,2016-01-01 00:00:00,3.05,279.28,277.97,91.4,200.0,0.0,14066001_0_0
1,14066001,2016-01-01 01:00:00,2.57,278.76,277.45,91.4,190.0,0.0,14066001_0_1
2,14066001,2016-01-01 02:00:00,2.26,278.27,277.02,91.7,181.0,0.0,14066001_0_2
3,14066001,2016-01-01 03:00:00,2.62,277.98,276.95,93.0,159.0,0.0,14066001_0_3
4,14066001,2016-01-01 04:00:00,2.99,277.32,276.72,95.9,171.0,0.0,14066001_0_4
...,...,...,...,...,...,...,...,...,...
4409469,95690001,2017-12-30 19:00:00,9.10,286.68,283.44,80.8,239.0,0.0,95690001_729_19
4409470,95690001,2017-12-30 20:00:00,8.58,286.39,283.21,81.1,231.0,0.0,95690001_729_20
4409471,95690001,2017-12-30 21:00:00,8.74,286.28,283.40,82.6,226.0,0.0,95690001_729_21
4409472,95690001,2017-12-30 22:00:00,9.04,286.21,283.29,82.4,224.0,0.0,95690001_729_22


Missing Data : 6899007 
 Shape is : (4409474, 9)


In [33]:
xtrain_obs_ll = xtrain_obs.merge(coords, how='left', on='number_sta')
display(xtrain_obs_ll)

,number_sta,date,ff,t,td,hu,dd,precip,Id,lat,lon,height_sta
0,14066001,2016-01-01 00:00:00,3.05,279.28,277.97,91.4,200.0,0.0,14066001_0_0,49.334,-0.431,2.0
1,14066001,2016-01-01 01:00:00,2.57,278.76,277.45,91.4,190.0,0.0,14066001_0_1,49.334,-0.431,2.0
2,14066001,2016-01-01 02:00:00,2.26,278.27,277.02,91.7,181.0,0.0,14066001_0_2,49.334,-0.431,2.0
3,14066001,2016-01-01 03:00:00,2.62,277.98,276.95,93.0,159.0,0.0,14066001_0_3,49.334,-0.431,2.0
4,14066001,2016-01-01 04:00:00,2.99,277.32,276.72,95.9,171.0,0.0,14066001_0_4,49.334,-0.431,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4409469,95690001,2017-12-30 19:00:00,9.10,286.68,283.44,80.8,239.0,0.0,95690001_729_19,49.108,1.831,126.0
4409470,95690001,2017-12-30 20:00:00,8.58,286.39,283.21,81.1,231.0,0.0,95690001_729_20,49.108,1.831,126.0
4409471,95690001,2017-12-30 21:00:00,8.74,286.28,283.40,82.6,226.0,0.0,95690001_729_21,49.108,1.831,126.0
4409472,95690001,2017-12-30 22:00:00,9.04,286.21,283.29,82.4,224.0,0.0,95690001_729_22,49.108,1.831,126.0


In [34]:
ytrain = pd.read_csv(ytrain_path, parse_dates=['date'], infer_datetime_format=True)
ytrain['number_sta'] = ytrain['number_sta'].astype('category')
display(ytrain)
print('Missing Data :', ytrain.isna().sum().sum(), '\n', 'Shape is :', ytrain.shape)

,date,number_sta,Ground_truth,Id
0,2016-01-02,14066001,3.4,14066001_0
1,2016-01-02,14126001,0.5,14126001_0
2,2016-01-02,14137001,3.4,14137001_0
3,2016-01-02,14216001,4.0,14216001_0
4,2016-01-02,14296001,13.3,14296001_0
...,...,...,...,...
183742,2017-12-31,86137003,5.0,86137003_729
183743,2017-12-31,86165005,3.2,86165005_729
183744,2017-12-31,86272002,1.8,86272002_729
183745,2017-12-31,91200002,1.6,91200002_729


Missing Data : 21640 
 Shape is : (183747, 4)


In [35]:
xtest_obs = pd.read_csv(xtest_obs_path,infer_datetime_format=True)
display(xtest_obs)
print('Missing Data :', xtest_obs.isna().sum().sum(), '\n', 'Shape is :', xtest_obs.shape)

,dd,hu,td,t,ff,precip,month,Id
0,NaN,NaN,NaN,278.35,NaN,NaN,12,14047002_277_4
1,NaN,NaN,NaN,278.40,NaN,0.0,12,14047002_277_5
2,NaN,NaN,NaN,279.01,NaN,0.0,12,14047002_277_6
3,NaN,NaN,NaN,279.66,NaN,0.0,12,14047002_277_7
4,NaN,NaN,NaN,279.99,NaN,0.0,12,14047002_277_8
...,...,...,...,...,...,...,...,...
2304797,190.0,82.8,277.00,279.74,10.62,0.0,12,95690001_176_19
2304798,195.0,84.2,277.44,279.93,11.86,0.0,12,95690001_176_20
2304799,199.0,85.7,277.95,280.21,11.77,0.0,12,95690001_176_21
2304800,198.0,85.3,278.25,280.58,10.16,0.0,12,95690001_176_22


Missing Data : 3914287 
 Shape is : (2304802, 8)


In [36]:
bltest_obs = pd.read_csv(bltest_obs_path,infer_datetime_format=True)

In [37]:
bltrain_for = pd.read_csv(bltrain_for_path, infer_datetime_format=True)
bltest_for = pd.read_csv(bltest_for_path, infer_datetime_format=True)

## Données forecast

In [38]:
model = 'arome'
param = "tp"
file_date = dt.datetime(2017, 2, 14) 
X_fore_2D = "C:/Users/Alexia/Desktop/Data/Train/X_forecast/2D_arome_20170214.nc"
data = xr.open_dataset(X_fore_2D)
data.to_netcdf("saved_on_disk.nc")

In [39]:
X_forecast_2D = xr.open_dataset("saved_on_disk.nc")

In [32]:
X_forecast_2D

NameError: name 'X_forecast_2D' is not defined

In [30]:
# ytrain_date = ytrain[(ytrain['date'] =='2017-02-14')]
# ytrain_date = ytrain_date.merge(coords, how='left', on='number_sta')
# display(ytrain_date)

,date,number_sta,Ground_truth,Id,lat,lon,height_sta
0,2017-02-14,14066001,0.0,14066001_409,49.334,-0.431,2.0
1,2017-02-14,14126001,0.0,14126001_409,49.145,0.042,125.0
2,2017-02-14,14137001,0.0,14137001_409,49.180,-0.456,67.0
3,2017-02-14,14216001,0.0,14216001_409,48.928,-0.149,155.0
4,2017-02-14,14296001,0.4,14296001_409,48.795,-1.037,336.0
...,...,...,...,...,...,...,...
247,2017-02-14,86137003,1.2,86137003_409,47.035,0.098,96.0
248,2017-02-14,86165005,1.4,86165005_409,46.412,0.841,153.0
249,2017-02-14,86272002,NaN,86272002_409,46.839,0.457,120.0
250,2017-02-14,91200002,0.0,91200002_409,48.526,1.993,116.0


In [43]:
#display(X_forecast_2D[param])

In [ ]:
xtrain_obs_ll.loc[:,"lon"]
X_forecast_2D[:,"longitude"]

In [ ]:
def lat_lon (xtrain,xforecast):#distance euclidienne 
    for i in range()
    xtrain.loc[:,"lon"] 
    xforecast.loc[:,"longitude"]
    xtrain.loc[:,"lat"]
    xforecast.loc[:,"latitude"]